# Calcular y usar PCA  (Análisis de Componentes Principales) en datos 2D
**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025  


### Gráficos interactivos

Para los gráficos se usa matplotlib:
```
pip install matplotlib
```

Para gráficos interactivos (por ej. hacer zoom):

  1. Instalar ipympl:  `pip install ipympl`
  2. Reiniciar jupyter 
  3. Reemplazar `%matplotlib inline` por `%matplotlib widget`


In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Descomentar esta linea para graficos interactivos
## %matplotlib widget


## Cargar conjunto de referencia R (datos de búsqueda)

In [ ]:
import numpy

dataset_r = numpy.array(
    [
        [41.94, 46.371],
        [38.565, 62.975],
        [64.492, 40.318],
        [49.356, 46.11],
        [94.171, 9.332],
        [37.839, 58.573],
        [72.596, 28.758],
        [66.03, 34.948],
        [98.65, 0.51],
        [92.75, 7.154],
        [15.849, 88.241],
        [75.305, 24.771],
        [44.565, 43.065],
        [18.742, 72.566],
        [81.323, 25.513],
        [62.491, 38.015],
        [62.775, 29.699],
        [53.259, 40.689],
        [88.158, 18.807],
        [21.801, 76.334],
        [83.757, 20.567],
        [44.154, 51.959],
        [22.309, 69.964],
        [92.945, 13.903],
        [70.989, 30.13],
        [72.531, 23.106],
        [40.964, 63.974],
        [33.935, 66.08],
        [90.294, 13.576],
        [17.279, 80.112],
        [7.616, 83.688],
        [70.368, 23.538],
        [78.898, 24.077],
        [61.504, 41.521],
        [28.986, 64.275],
        [19.286, 83.615],
        [59.819, 34.649],
        [87.192, 14.297],
        [30.06, 71.112],
        [32.858, 68.683],
        [88.202, 16.878],
        [25.043, 71.219],
        [74.758, 31.516],
        [26.185, 58.557],
        [94.252, 1.778],
        [36.64, 69.521],
        [80.523, 18.94],
        [87.195, 11.386],
        [85.189, 13.871],
        [35.3, 62.118],
    ]
)

print("Dataset R={}".format(dataset_r.shape))

In [ ]:
import matplotlib.pyplot as plt


def dibujar_dataset(datar, dataq=None):
    x = datar[:, 0]
    y = datar[:, 1]
    numpy.random.seed(5)
    plt.scatter(x, y, color=numpy.random.rand(datar.shape[0], 3))
    if dataq is not None:
        xq = dataq[:, 0]
        yq = dataq[:, 1]
        plt.scatter(xq, yq, label="datos_q", color="blue", marker="^", s=100)
        plt.legend()
    plt.show()


dibujar_dataset(dataset_r)

## Centrar conjunto R

In [ ]:
promedios = dataset_r.mean(axis=0)
print("promedio: {}".format(promedios))

datos_centrados = dataset_r - promedios
dibujar_dataset(datos_centrados)

## Matriz de covarianza del conjunto centrado

In [ ]:
# se usa transpose() para que calcule una matriz de 128x128, bias=False para varianzas divididas por n-1
matriz_covarianza = numpy.cov(datos_centrados.transpose(), bias=False)
print("matriz de covarianza: {}".format(matriz_covarianza.shape))
print(matriz_covarianza)

## Valores y vectores propios de la matriz de covarianza

In [ ]:
eigenvalues, eigenvectors = numpy.linalg.eig(matriz_covarianza)
print("eigenvalues: {}".format(eigenvalues.shape))
print(eigenvalues)
print("eigenvectors: {}".format(eigenvectors.shape))
print(eigenvectors)

# Ordenar valores propios de de mayor a menor

In [ ]:
# indices para ordenar
indices_menor_a_mayor = eigenvalues.argsort()
indices_mayor_a_menor = indices_menor_a_mayor[::-1]

# obtener valores y vectores ordenados de mayor a menor
eigenvalues_sorted = eigenvalues[indices_mayor_a_menor]
eigenvectors_sorted = eigenvectors[:, indices_mayor_a_menor]

for i in range(len(eigenvalues_sorted)):
    print("componente principal #{}".format(i))
    print("    varianza = {:.2f}".format(eigenvalues_sorted[i]))
    print("    vector   = {}".format(eigenvectors_sorted[:, i]))

## Dibujar componentes principales

In [ ]:
def dibujar_nuevos_ejes(datar, vector1, vector2, dataq=None):
    x = datar[:, 0]
    y = datar[:, 1]
    numpy.random.seed(5)
    plt.scatter(x, y, color=numpy.random.rand(datar.shape[0], 3))
    plt.quiver(0, 0, vector1[0], vector1[1], color="red", scale=5)
    plt.quiver(0, 0, vector2[0], vector2[1], color="magenta", scale=5)
    plt.xlim(-70, 70)
    plt.ylim(-70, 70)
    if dataq is not None:
        xq = dataq[:, 0]
        yq = dataq[:, 1]
        plt.scatter(xq, yq, label="datos_q", color="blue", marker="^", s=100)
        plt.legend()
    plt.show()


vector1 = eigenvectors_sorted[:, 0]
vector2 = eigenvectors_sorted[:, 1]
dibujar_nuevos_ejes(datos_centrados, vector1, vector2)

## Definir matriz de proyección (matriz de vectores propios) y rotar ejes

In [ ]:
matriz_transformacion = eigenvectors_sorted

datos_transformados = datos_centrados.dot(matriz_transformacion)

vector1_transformado = vector1.dot(matriz_transformacion)
vector2_transformado = vector2.dot(matriz_transformacion)

# con la transformacion los eigenvector serán los ejes (1,0) y (0,1)
print("vector1_transformado={}".format(vector1_transformado))
print("vector2_transformado={}".format(vector2_transformado))

dibujar_nuevos_ejes(datos_transformados, vector1_transformado, vector2_transformado)

## Reducir dimensiones (convertir 2d a 1d quitando el eje de menor valor propio)

In [ ]:
datos_proyectados1d = numpy.copy(datos_transformados)
datos_proyectados1d[:, 1] = 0
dibujar_nuevos_ejes(datos_proyectados1d, vector1_transformado, vector2_transformado)

## Opcional: Proyección inversa (espacio 1d a espacio 2d)

In [ ]:
# invertir la matriz
# notar que la inversa es la matriz transpose: matriz_transformacion.T
transformacion_inversa = numpy.linalg.inv(matriz_transformacion)

datos_revertidos = datos_proyectados1d.dot(transformacion_inversa) + promedios

dibujar_dataset(datos_revertidos)

# Búsqueda en espacios reducidos con PCA

## Conjunto de datos de consulta Q

In [ ]:
dataset_q = numpy.array(
    [
        [68.184, 36.637],
        [77.422, 19.473],
        [18.947, 82.306],
        [25.554, 65.87],
        [91.477, 10.556],
        [41.624, 52.677],
    ]
)

print("Dataset Q={}".format(dataset_q.shape))

dibujar_dataset(dataset_r, dataset_q)

## Restar promedio a Q (usar el mismo centrado de los datos)

In [ ]:
datos_q_centrados = dataset_q - promedios

dibujar_nuevos_ejes(datos_centrados, vector1, vector2, datos_q_centrados)

## Rotar el conjunto de consulta

Notar que los vecinos más cercanos no cambian porque es solo un cambio de ejes.

In [ ]:
datos_q_transformados = datos_q_centrados.dot(matriz_transformacion)

dibujar_nuevos_ejes(
    datos_transformados,
    vector1_transformado,
    vector2_transformado,
    datos_q_transformados,
)

## Reducir la dimensión de Q (quitar coordenada de menor valor propio)

In [ ]:
datos_q_proyectados1d = datos_q_transformados
datos_q_proyectados1d[:, 1] = 0

dibujar_nuevos_ejes(
    datos_proyectados1d,
    vector1_transformado,
    vector2_transformado,
    datos_q_proyectados1d,
)

## Comparar vecinos más cercanos entre espacio original y proyeccion 1d

In [ ]:
from scipy.spatial import distance


def comparar_cercanos(dataset_q, dataset_r, dataset_q_proyeccion, dataset_r_proyeccion):
    # busqueda lineal
    distancias_real = distance.cdist(dataset_q, dataset_r, metric="euclidean")
    distancias_proyeccion = distance.cdist(
        dataset_q_proyeccion, dataset_r_proyeccion, metric="euclidean"
    )
    # posicion del mas cercano
    posicion_min_real = numpy.argmin(distancias_real, axis=1)
    posicion_min_proyeccion = numpy.argmin(distancias_proyeccion, axis=1)
    # comparar
    bien = 0
    for i in range(dataset_q.shape[0]):
        print("Query #{}".format(i))
        query = dataset_q[i]
        posicion_nn = posicion_min_real[i]
        nn = dataset_r[posicion_nn]
        print("    ORIGINAL  : q={} -> NN=#{} r={}".format(query, posicion_nn, nn))
        query_proyeccion = dataset_q_proyeccion[i]
        posicion_nn_proyeccion = posicion_min_proyeccion[i]
        proyeccion_nn = dataset_r_proyeccion[posicion_nn_proyeccion]
        igual = "¡distinto!"
        if posicion_nn == posicion_nn_proyeccion:
            bien += 1
            igual = "igual NN"
        print(
            "    PROYECCION: q={}  -> NN=#{} r={}  ({})".format(
                query_proyeccion, posicion_nn_proyeccion, proyeccion_nn, igual
            )
        )
    # resultado global
    total = dataset_q.shape[0]
    correctas = bien / total
    print()
    print("precision del NN={:.1f}% ({} de {})".format(correctas * 100, bien, total))


# descarto la segunda coordenada que vale 0
dataset_q_1d = datos_q_proyectados1d[:, 0:-1]
dataset_r_1d = datos_proyectados1d[:, 0:-1]

comparar_cercanos(dataset_q, dataset_r, dataset_q_1d, dataset_r_1d)